In [1]:
# Install DBT with PostgreSQL support
!pip install dbt-postgres


In [2]:
%cd C:/Users/Bob/Desktop/10Acadamy/week7/medical-data-warehouse


C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse


In [3]:
!dbt init medical_dbt


07:08:01  Running with dbt=1.10.4
07:08:01  A project called medical_dbt already exists here.


In [4]:
profile_config = """
medical_dbt:
  target: dev
  outputs:
    dev:
      type: postgres
      host: localhost
      user: warehouse
      password: drax
      port: 5432
      dbname: telegram_db
      schema: public
      threads: 4
      timeout_seconds: 300
"""

import os

os.makedirs(os.path.expanduser("~/.dbt"), exist_ok=True)

with open(os.path.expanduser("~/.dbt/profiles.yml"), "w") as f:
    f.write(profile_config)

print("✅ profiles.yml created at ~/.dbt/profiles.yml")


✅ profiles.yml created at ~/.dbt/profiles.yml


In [5]:
# Test the connection to your Postgres database
!dbt debug --project-dir medical_dbt


07:08:06  Running with dbt=1.10.4
07:08:06  dbt version: 1.10.4
07:08:06  python version: 3.13.3
07:08:06  python path: c:\Users\Bob\Desktop\10Acadamy\week7\.venv\Scripts\python.exe
07:08:06  os info: Windows-11-10.0.26100-SP0
07:08:06  Using profiles dir at C:\Users\Bob\.dbt
07:08:06  Using profiles.yml file at C:\Users\Bob\.dbt\profiles.yml
07:08:06  Using dbt_project.yml file at medical_dbt\dbt_project.yml
07:08:06  adapter type: postgres
07:08:06  adapter version: 1.9.0
07:08:07  Configuration:
07:08:07    profiles.yml file [OK found and valid]
07:08:07    dbt_project.yml file [OK found and valid]
07:08:07  Required dependencies:
07:08:07   - git [OK found]

07:08:07  Connection:
07:08:07    host: localhost
07:08:07    port: 5432
07:08:07    user: warehouse
07:08:07    database: telegram_db
07:08:07    schema: public
07:08:07    connect_timeout: 10
07:08:07    role: None
07:08:07    search_path: None
07:08:07    keepalives_idle: 0
07:08:07    sslmode: None
07:08:07    sslcert: None

In [6]:
stg_model = """
with source as (
    select * from {{ source('public', 'raw_telegram_messages') }}
),

renamed as (
    select
        message_id::bigint as message_id,
        date::timestamp as message_date,
        sender_id::text as sender_id,
        text::text as message_text,
        channel::text as channel,
        has_photo::boolean as has_photo
    from source
)

select * from renamed
"""

models_path = "medical_dbt/models/staging"
os.makedirs(models_path, exist_ok=True)

with open(f"{models_path}/stg_telegram_messages.sql", "w") as f:
    f.write(stg_model)

print("✅ Created staging model: stg_telegram_messages.sql")


✅ Created staging model: stg_telegram_messages.sql


✅ CELL 6 – Register your source
In medical_dbt/models, create a file called schema.yml:

In [7]:
schema_yml = """
version: 2

sources:
  - name: public
    database: telegram_db
    schema: public
    tables:
      - name: raw_telegram_messages
"""

with open("medical_dbt/models/schema.yml", "w") as f:
    f.write(schema_yml)

print("✅ schema.yml created with raw_telegram_messages as source")


✅ schema.yml created with raw_telegram_messages as source


✅ CELL 7 – Run the DBT model

In [8]:
!dbt run --select stg_telegram_messages --project-dir medical_dbt


07:08:14  Running with dbt=1.10.4
07:08:14  Registered adapter: postgres=1.9.0
07:08:16  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.medical_dbt.example
07:08:17  Found 1 test, 4 models, 1 source, 433 macros
07:08:17  
07:08:17  Concurrency: 4 threads (target='dev')
07:08:17  
07:08:17  1 of 1 START sql view model public.stg_telegram_messages ....................... [RUN]
07:08:17  1 of 1 OK created sql view model public.stg_telegram_messages .................. [CREATE VIEW in 0.18s]
07:08:17  
07:08:17  Finished running 1 view model in 0 hours 0 minutes and 0.60 seconds (0.60s).
07:08:17  
07:08:17  Completed successfully
07:08:17  
07:08:17  Done. PASS=1 WARN=0 ERROR=0 SKIP=0 NO-OP=0 TOTAL=1


✅ CELL 8 – Create dim_channels model



In [9]:
dim_channels_sql = """
with base as (
    select
        distinct channel
    from {{ ref('stg_telegram_messages') }}
),

dim_channels as (
    select
        row_number() over (order by channel) as channel_id,
        channel
    from base
)

select * from dim_channels
"""

with open("medical_dbt/models/dim_channels.sql", "w") as f:
    f.write(dim_channels_sql)

print("✅ dim_channels.sql created")


✅ dim_channels.sql created


CELL 9 – Create dim_dates model

In [10]:
dim_dates_sql = """
with raw_dates as (
    select
        distinct message_date::date as date
    from {{ ref('stg_telegram_messages') }}
),

dim_dates as (
    select
        row_number() over (order by date) as date_id,
        date,
        extract(year from date) as year,
        extract(month from date) as month,
        extract(day from date) as day,
        to_char(date, 'Day') as day_name
    from raw_dates
)

select * from dim_dates
"""

with open("medical_dbt/models/dim_dates.sql", "w") as f:
    f.write(dim_dates_sql)

print("✅ dim_dates.sql created")


✅ dim_dates.sql created


✅ CELL 10 – Create fct_messages (fact table)


In [ ]:
fct_messages_sql = """
with base as (
    select
        msg.message_id,
        msg.sender_id,
        msg.message_text,
        msg.has_photo,
        msg.channel,
        msg.message_date::date as post_date  -- ✅ Renamed alias
    from {{ ref('stg_telegram_messages') }} msg
),

joined as (
    select
        base.message_id,
        base.sender_id,
        base.message_text,
        base.has_photo,
        dc.channel_id,
        dd.date_id,
        base.post_date,  -- ✅ Include it in select
        length(base.message_text) as message_length
    from base
    left join {{ ref('dim_channels') }} dc on base.channel = dc.channel
    left join {{ ref('dim_dates') }} dd on base.post_date = dd.date  -- ✅ Use updated alias
)

select * from joined
"""

with open("medical_dbt/models/fct_messages.sql", "w") as f:
    f.write(fct_messages_sql)

print("✅ fct_messages.sql created")


✅ fct_messages.sql created


✅ CELL 11 – Add tests to schema.yml
Update your existing schema.yml inside medical_dbt/models:

In [12]:
schema_yml = """
version: 2

sources:
  - name: public
    database: telegram_db
    schema: public
    tables:
      - name: raw_telegram_messages

models:
  - name: stg_telegram_messages
    columns:
      - name: message_id
        tests:
          - unique
          - not_null

  - name: dim_channels
    columns:
      - name: channel_id
        tests:
          - unique
          - not_null

  - name: dim_dates
    columns:
      - name: date_id
        tests:
          - unique
          - not_null

  - name: fct_messages
    columns:
      - name: message_id
        tests:
          - unique
          - not_null
"""

with open("medical_dbt/models/schema.yml", "w") as f:
    f.write(schema_yml)

print("✅ schema.yml updated with tests")


✅ schema.yml updated with tests


✅ CELL 12 – Create a Custom Test
Create a new file inside tests/ folder (you can create the folder if it doesn't exist):

In [13]:
custom_test_sql = """
-- Ensure that no message is missing a date
select *
from {{ ref('fct_messages') }}
where date_id is null
"""

os.makedirs("medical_dbt/tests", exist_ok=True)
with open("medical_dbt/tests/no_null_dates.sql", "w") as f:
    f.write(custom_test_sql)

print("✅ Custom test 'no_null_dates.sql' created")


✅ Custom test 'no_null_dates.sql' created


✅ CELL 13 – Run All Models

In [14]:
!dbt run --project-dir medical_dbt


07:08:22  Running with dbt=1.10.4
07:08:22  Registered adapter: postgres=1.9.0
07:08:23  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.medical_dbt.example
07:08:23  Found 9 data tests, 4 models, 1 source, 433 macros
07:08:23  
07:08:23  Concurrency: 4 threads (target='dev')
07:08:23  
07:08:23  1 of 4 START sql view model public.stg_telegram_messages ....................... [RUN]
07:08:24  1 of 4 OK created sql view model public.stg_telegram_messages .................. [CREATE VIEW in 0.17s]
07:08:24  2 of 4 START sql view model public.dim_channels ................................ [RUN]
07:08:24  3 of 4 START sql view model public.dim_dates ................................... [RUN]
07:08:24  2 of 4 OK created sql view model public.dim_channels ........................... [CREATE VIEW in 0.12s]
07:08:24  3 of 4 OK created sql view model public.dim_dates .............................

✅ CELL 14 – Run Tests

In [15]:
!dbt test --project-dir medical_dbt


07:08:28  Running with dbt=1.10.4
07:08:29  Registered adapter: postgres=1.9.0
07:08:29  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.medical_dbt.example
07:08:29  Found 9 data tests, 4 models, 1 source, 433 macros
07:08:29  
07:08:29  Concurrency: 4 threads (target='dev')
07:08:29  
07:08:30  1 of 9 START test no_null_dates ................................................ [RUN]
07:08:30  3 of 9 START test not_null_dim_dates_date_id ................................... [RUN]
07:08:30  4 of 9 START test not_null_fct_messages_message_id ............................. [RUN]
07:08:30  2 of 9 START test not_null_dim_channels_channel_id ............................. [RUN]
07:08:30  1 of 9 PASS no_null_dates ...................................................... [PASS in 0.25s]
07:08:30  4 of 9 PASS not_null_fct_messages_message_id ................................... [PASS in 0.25s]
07:08:

✅ CELL 15 – Generate Documentation



In [16]:
# Generate and view docs
!dbt docs generate --project-dir medical_dbt
!dbt docs serve --project-dir medical_dbt


07:08:35  Running with dbt=1.10.4
07:08:35  Registered adapter: postgres=1.9.0
07:08:36  [WARNING]: Configuration paths exist in your dbt_project.yml file which do not apply to any resources.
There are 1 unused configuration paths:
- models.medical_dbt.example
07:08:36  Found 9 data tests, 4 models, 1 source, 433 macros
07:08:36  
07:08:36  Concurrency: 4 threads (target='dev')
07:08:36  
07:08:37  Building catalog
07:08:37  Catalog written to C:\Users\Bob\Desktop\10Acadamy\week7\medical-data-warehouse\medical_dbt\target\catalog.json
^C
